### Importa as bibliotecas a serem usadas

In [39]:
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import math
from os import mkdir, path

### Define caminho onde os dados .VEC estao salvos e cria uma pasta Dados_csv, onde amarzenara os dados gerados pelo script

In [40]:
caminho = Path('dados/tidi/')
arqs = sorted([str(i) for i in caminho.glob('**/*.VEC')])
arquivos_csv = Path('dados/tidi/dados_csv')
arquivos_csv.mkdir(exist_ok=True)

In [41]:
if not path.exists('dados/tidi/dados_zonal'):
    mkdir('dados/tidi/dados_zonal')
    mkdir('dados/tidi/dados_meridional')

### Loop para cada arquivo .VEC
### df.to_csv salva os dados

In [42]:
for arquivo in arqs:
    str_arq = str(arquivo)
    # if 'D010_R01' not in str_arq:
    #     continue

    # comando para ler arquivo no formato netcdf
    df=xr.open_dataset(arquivo, engine='scipy')
    df= df.to_dataframe()
    #selecionando variaveis
    df2=df[['ascending','alt_retrieved','ut_date','ut_time','lat','lon','data_ok','u','var_u','v','var_v']]
    #selecionando latitude de São João do Cariri
    df2=df2.loc[df['lat'] >= -17]
    df2=df2.loc[df['lat'] <= 3]
    #selecionando longitude de São João do Cariri
    df2=df2.loc[df['lon'] >= 314]
    df2=df2.loc[df['lon'] <= 334]
    #Selecionando o intervalo de altitude
    df2=df2.loc[df['alt_retrieved'] >= 70]
    df2=df2.loc[df['alt_retrieved'] <= 100]
    df.ut_time
    time2=df.ut_time
    #convertendo o tempo para unidade de hora
    time2=time2/1e3/6e1/6e1
    df2['UT_time']=time2

    df2=df2[['ascending','alt_retrieved','ut_date','UT_time','lat','lon','data_ok','u','var_u','v','var_v']]
    df2
    # comando para salvar os arquivos
    #your_name= nome para salvar os arquivos
    df2.to_csv(f'dados/tidi/dados_csv/{str(arquivo)[16:-4]}.csv', sep=';')

In [43]:
df2.head()

ascending  alt_retrieved     ut_date    UT_time        lat  \
nalts nvec                                                              
0     240       b'F'           70.0  b'2006365'   4.667778  -0.630595   
      241       b'F'           70.0  b'2006365'   4.703889  -8.046996   
      242       b'F'           70.0  b'2006365'   4.740000 -15.528027   
      858       b'T'           70.0  b'2006365'  18.403889 -11.780641   
      859       b'T'           70.0  b'2006365'  18.439722  -4.329545   

                   lon data_ok   u  var_u   v  var_v  
nalts nvec                                            
0     240   322.096771    b'T' NaN    NaN NaN    NaN  
      241   323.596558    b'T' NaN    NaN NaN    NaN  
      242   324.876801    b'T' NaN    NaN NaN    NaN  
      858   325.391876    b'T' NaN    NaN NaN    NaN  
      859   326.779785    b'T' NaN    NaN NaN    NaN

### Filtrando os arquivos csvs do radar que possuem mais dados para cada dia

In [44]:
caminho = Path('dados/tidi/dados_csv/')
arquivos = sorted([i for i in caminho.glob('*.csv')])

In [45]:
dict_count_null_files = {}

for arquivo in arquivos:
    df = pd.read_csv(arquivo, sep=';')
    count_nulls = df.isna().sum().sum()
    
    key = str(arquivo)[21:-25]
    value = (arquivo, count_nulls)
    
    if key in dict_count_null_files:
        dict_count_null_files[key] += [value]
    else:
         dict_count_null_files[key] = [value]

In [46]:
files = []
for key in dict_count_null_files:
    if len(dict_count_null_files[key]) > 1:
        min = 999999 
        for i, v in enumerate(dict_count_null_files[key]):
            if v[1] < min:
                min = v[1]
                f_min  = v[0]
        files.append(f_min)

In [47]:
len(files)

728

In [48]:
files

[WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005001_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005002_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005003_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005004_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005005_P0100_S0450_D011_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005006_P0100_S0450_D011_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005007_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005008_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005009_P0100_S0450_D011_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005010_P0100_S0200_D007_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005011_P0100_S0450_D011_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005012_P0100_S0450_D011_R01.csv'),
 WindowsPath('dados/tidi/dados_csv/TIDI_PB_2005013_P

### Converter dados csv para txt com ventos zonal e meridional

<strong> u = Zonal and v = Meridional

<strong> arquivo final: zonal/meridional_yyyymmdd <br>
<strong> colunas do arquivo final: tempo_corrido, 70km ... 100km, std_70km ... std_100km 

In [49]:
dict_months = {
    '01': [1, 31],
    '02': [32, 59],
    '03': [60, 90],
    '04': [91, 120],
    '05': [121, 151],
    '06': [152, 181],
    '07': [182, 212],
    '08': [213, 243],
    '09': [244, 273],
    '10': [274, 304],
    '11': [305, 334],
    '12': [335, 365],
}

dict_day_to_month = { i:k  for k,v in zip(dict_months.keys(), dict_months.values()) for i in range(v[0], v[1]+1)}

In [50]:
dict_day_to_month[1]

'01'

In [51]:
def define_listas_df(df):
    lista_alt = list(df.alt_retrieved.value_counts().index)
    lista_alt_df = [str(i)+'km' for i in lista_alt]
    lista_alt_std_df = ['std_'+str(i)+'km' for i in lista_alt]
    lista_nvecs = list(df.nvec.value_counts().index)

    return lista_alt_df, lista_alt_std_df, lista_nvecs

In [52]:
def retorna_df_convertido(df, lista_alt_df, lista_alt_std_df, lista_nvecs, dir_vento):
    if lista_alt_df: 
        df_converted = pd.DataFrame(columns=['tempo_decorrido']+lista_alt_df+lista_alt_std_df, index=range(0, len(lista_nvecs)))
        
        for vec,i in zip(lista_nvecs, range(len(lista_nvecs))):
            df_ = df[df.nvec==vec][['UT_time', 'alt_retrieved', dir_vento, 'var_'+dir_vento]].reset_index(drop=True).T
            for col, alt_df, alt_std_df in zip(df_.columns, lista_alt_df, lista_alt_std_df):
                df_converted.iloc[i]['tempo_decorrido'] = df_[col]['UT_time']
                df_converted.iloc[i][alt_df] = df_[col][dir_vento]
                df_converted.iloc[i][alt_std_df] = math.sqrt(df_[col]['var_'+dir_vento])
    else:
        df_converted = pd.DataFrame()
          
    return df_converted
    

In [53]:
for key in dict_count_null_files:
    dfs = []
    l_paths = [i[0] for i in dict_count_null_files[key]]
    
    for path in l_paths:
        dfs.append(pd.read_csv(path, sep=';'))
    
    df = pd.concat(dfs, ignore_index=True).reset_index(drop=True)
    
    ano = str(path)[29:33]
    dia = str(path)[33:36]
    mes = dict_day_to_month[int(dia)]

    lista_alt_df, lista_alt_std_df, lista_nvecs = define_listas_df(df)
  
    for dir_vento, nome_vento in zip(['u', 'v'], ['zonal', 'meridional']):
        df_convertido = retorna_df_convertido(df, lista_alt_df, lista_alt_std_df, lista_nvecs, dir_vento)
        print(nome_vento+'_'+ano+mes+'_'+dia)
        if df_convertido.empty:
            continue
        else:
            df_convertido.to_csv(f'dados/tidi/dados_{nome_vento}/dados_{nome_vento}_{ano}{mes}_{dia}.txt', ';', index=False)

zonal_200501_001
meridional_200501_001
zonal_200501_002
meridional_200501_002
zonal_200501_003
meridional_200501_003
zonal_200501_004
meridional_200501_004
zonal_200501_005
meridional_200501_005
zonal_200501_006
meridional_200501_006
zonal_200501_007
meridional_200501_007
zonal_200501_008
meridional_200501_008
zonal_200501_009
meridional_200501_009
zonal_200501_010
meridional_200501_010
zonal_200501_011
meridional_200501_011
zonal_200501_012
meridional_200501_012
zonal_200501_013
meridional_200501_013
zonal_200501_014
meridional_200501_014
zonal_200501_015
meridional_200501_015
zonal_200501_016
meridional_200501_016
zonal_200501_017
meridional_200501_017
zonal_200501_018
meridional_200501_018
zonal_200501_019
meridional_200501_019
zonal_200501_020
meridional_200501_020
zonal_200501_021
meridional_200501_021
zonal_200501_022
meridional_200501_022
zonal_200501_023
meridional_200501_023
zonal_200501_024
meridional_200501_024
zonal_200501_025
meridional_200501_025
zonal_200501_026
meridion